In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
#from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations
import idaes
from itertools import permutations, product

from fim_doe import *

import pyomo.common.unittest as unittest

In [2]:
import cvxpy as cp

from reactor_kinetics import *

ModuleNotFoundError: No module named 'cvxpy'

## FIM generator

In [ ]:
def generate_exp(t_set, CA0, T):  
    '''Generate experiments. 
    t_set: time control set for T.
    CA0: CA0 value
    T: A list of T 
    '''
    assert(len(t_set)==len(T)), 'T should have the same length as t_set'
    
    T_con_initial = {}
    for t, tim in enumerate(t_set):
        T_con_initial[tim] = T[t]
        
    dv_dict_overall = {'CA0': {0: CA0},'T': T_con_initial}
    return dv_dict_overall

In [ ]:
def return_FIM(measure_pass):
    # Create model function
    createmod = create_model

    # discretization by Pyomo.DAE
    disc = disc_for_measure

    # Control time set [h]
    t_control = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]

    

    # design variable and its control time set
    dv_pass = {'CA0': [0],'T': t_control}

    t_test = [0.125, 0.375, 0.625, 0.875]

    
    measure_class =  Measurements(measure_pass)

    # Define parameter nominal value 
    parameter_dict = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}
    
    # empty prior
    prior_all = np.zeros((4,4))

    prior_pass=np.asarray(prior_all)

    print('The prior information FIM:', prior_pass)
    

    # choose from 'sequential_finite', 'direct_kaug'
    # 'sequential_sipopt', 'sequential_kaug' is also available
    sensi_opt = 'sequential_finite'
    #sensi_opt = 'direct_kaug'

    # model option
    if sensi_opt == 'direct_kaug':
        args_ = [False]
    else:
        args_ = [True]


    # Define experiments
    exp1 = generate_exp(t_control, 5, [570, 300, 300, 300, 300, 300, 300, 300, 300])
    
    doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass, discretize_model=disc, args=args_, verbose=False)


    result = doe_object.compute_FIM(exp1,mode=sensi_opt, FIM_store_name = 'dynamic.csv', scale_nominal_param_value=True, 
                                    store_output = 'store_output', read_output=None,tee_opt=False,
                                    formula='central')


    result.calculate_FIM(doe_object.design_values)
    
    
    print('======Result summary======')
    print('Four design criteria log10() value:')
    print('A-optimality:', np.log10(result.trace))
    print('D-optimality:', np.log10(result.det))
    print('E-optimality:', np.log10(result.min_eig))
    print('Modified E-optimality:', np.log10(result.cond))
    
    return result.FIM, result.jaco_information

In [ ]:
# Create measurement object
# Measurement time points [h]
t_measure = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]

measure_pass = {'C':{'CA': t_measure, 'CB': t_measure, 'CC': t_measure}}
#measure_pass = {'C':{'CC': t_test}}

In [ ]:
fim1, jaco1 = return_FIM(measure_pass)
print(jaco1)

### Jacobian generator

In [ ]:
jaco = pd.DataFrame(jaco1)
jaco.to_csv('./kinetics_fim/Q.csv')

### FIM generator

In [ ]:
FIM_set = []

def to_dataframe(fim_list):
    
    fim_dict = {'P1': fim_list[0], 
               'P2': fim_list[1],
               'P3': fim_list[2], 
               'P4': fim_list[3]
               }
    fim_frame = pd.DataFrame(fim_dict)
    
    return fim_frame

In [ ]:
t_measure = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
#t_measure = [0]

#measure_pass = {'C':{'CA': t_measure, 'CB': t_measure, 'CC': t_measure}}
#measure_pass = {'C':{'CC': t_test}}

for idx, i in enumerate(t_measure):
    measure_pass = {'C':{'CA': [i]}}
    fim1, _ = return_FIM(measure_pass)
    FIM_set.append(fim1)
    print('FIM for CA, time ', i, ':', fim1)
    fim1_dataframe = to_dataframe(fim1)
    fim1_dataframe.to_csv('./kinetics_fim/CA_'+str(idx)+'.csv')
    
for idx, i in enumerate(t_measure):
    measure_pass = {'C':{'CB': [i]}}
    fim1, _ = return_FIM(measure_pass)
    FIM_set.append(fim1)
    print('FIM for CB, time ', i, ':', fim1)
    fim1_dataframe = to_dataframe(fim1)
    fim1_dataframe.to_csv('./kinetics_fim/CB_'+str(idx)+'.csv')
    
for idx, i in enumerate(t_measure):
    measure_pass = {'C':{'CC': [i]}}
    fim1, _ = return_FIM(measure_pass)
    FIM_set.append(fim1)
    print('FIM for CC, time ', i, ':', fim1)
    fim1_dataframe = to_dataframe(fim1)
    fim1_dataframe.to_csv('./kinetics_fim/CC_'+str(idx)+'.csv')

In [ ]:
print(np.shape(FIM_set))

print(FIM_set[0][1,1])

print(FIM_set)

## FIM reader

In [ ]:
def dataframe_return(data):
    fim1 = []
    fim1.append(data['P1'].to_list())
    fim1.append(data['P2'].to_list())
    fim1.append(data['P3'].to_list())
    fim1.append(data['P4'].to_list())
    
    return fim1 

In [ ]:
t_measure = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]

#t_measure = [0.125]



def FIM_reader(rank='onebyone'):
    
    FIM_set = []
    
    if rank=='onebyone':
        for idx, i in enumerate(t_measure):
            fim1_data = pd.read_csv('./kinetics_fim/CA_' + str(idx)+'.csv')
            
            fim1 = dataframe_return(fim1_data)
            FIM_set.append(fim1)
            
            fim2_data = pd.read_csv('./kinetics_fim/CB_' + str(idx)+'.csv')
            
            fim2 = dataframe_return(fim2_data)
            FIM_set.append(fim2)
            
            fim3_data = pd.read_csv('./kinetics_fim/CC_' + str(idx)+'.csv')
            
            fim3 = dataframe_return(fim3_data)
            FIM_set.append(fim3)
            
    elif rank=='onealong':
        for idx, i in enumerate(t_measure):
            fim1_data = pd.read_csv('./kinetics_fim/CA_' + str(idx)+'.csv')
            
            fim1 = dataframe_return(fim1_data)
            FIM_set.append(fim1)
            
        for idx, i in enumerate(t_measure):
            fim2_data = pd.read_csv('./kinetics_fim/CB_' + str(idx)+'.csv')
            
            fim2 = dataframe_return(fim2_data)
            FIM_set.append(fim2)
            
        for idx, i in enumerate(t_measure):
            fim3_data = pd.read_csv('./kinetics_fim/CC_' + str(idx)+'.csv')
            
            fim3 = dataframe_return(fim3_data)
            FIM_set.append(fim3)
            
            
    return FIM_set
        
FIM_set = FIM_reader(rank='onealong')
print(np.shape(FIM_set))
print(FIM_set[0][1][1])

In [ ]:
TotalF = np.zeros((4,4))
for i in range(4):
    for j in range(4):
        TotalF[i,j] = sum(FIM_set[k][i][j] for k in range(27))
        
print(TotalF)

print(np.trace(TotalF))
print(np.linalg.det(TotalF))
print(np.linalg.eigvals(TotalF))

### Jacobian reader

In [3]:
jaco_info = pd.read_csv('./kinetics_fim/Q.csv')

#print(jaco_info)


Q_stack = []

for i in range(27):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    
    Q_stack.append(Q_res)
    
print(Q_stack)

[[0.0, 0.0, 0.0, 0.0], [-1.346695417017152, -1.665334536937735e-13, 2.223379846162832, -1.6653345369377348e-13], [-1.039297035228287, -2.220446049250313e-13, 2.004516103508741, -1.6653345369377348e-13], [-0.7733130998612002, -1.3877787807814457e-13, 1.63790005416059, -1.1102230246251565e-13], [-0.560819154400885, -1.249000902703301e-13, 1.2648388615049688, -6.938893903907228e-14], [-0.3990444064009696, -4.8572257327350605e-14, 0.9415433202331, -2.081668171172168e-14], [-0.2797955547643657, -5.5511151231257833e-14, 0.6830391077798161, -2.081668171172168e-14], [-0.1939053002679436, -3.1225022567582534e-14, 0.486126099753268, -3.4694469519536138e-15], [-0.1331100105811684, -1.3877787807814455e-14, 0.3409191457896289, 1.040834085586084e-14], [0.0, 0.0, 0.0, 0.0], [0.0199157340269318, -1.3752037136890438, -0.0468172929494192, 4.379651748244728], [-0.2728277373608145, -1.38740313152641, 0.1675143115261734, 4.866408693837143], [-0.4952849992631014, -1.4015562993399122, 0.4972483613988609, 5.3

### FIM verificator

In [ ]:
#optimize_sol = [ 0, 0, 0, 
#                1, 1, 1, 1, 1, 1,
#                1, 1, 1, 1, 1, 1, 
#                0, 1, 1, 0, 1, 1,
#                0, 1, 1, 0, 1, 1]


SelectF = np.zeros((4,4))
for i in range(4):
    for j in range(4):
        SelectF[i,j] = sum(FIM_set[k][i,j]*yi for k, yi in enumerate(optimize_sol))
        
print(SelectF)

print(np.trace(SelectF))
print(np.linalg.det(SelectF))
print(np.linalg.eigvals(SelectF))

## CVXPY problem

no variance

In [ ]:
no_res = 27

In [ ]:
def eval_fim(yi):
     
    fim = sum(yy*FIM_set[idx] for idx, yy in enumerate(yi))
    #fim = cp.bmat(FIM_set)
        
    return fim 

def a_opt(yi):
    fim = eval_fim(yi)
    #return cp.matrix_frac(np.identity(fim.shape==[0]), fim)
    return cp.trace(fim)
    
def d_opt(yi):
    fim = eval_fim(yi)
    return cp.log_det(fim)

def e_opt(yi):
    fim = eval_fim(yi)
    return -cp.lambda_min(fim)

In [ ]:
yi = cp.Variable(no_res, nonneg=True)

p_cons = [cp.sum(yi) <= 20]

for i in yi:
    p_cons += [i<=1]

obj = cp.Maximize(d_opt(yi))

problem = cp.Problem(obj, p_cons)

In [ ]:
print('Optimal value:', problem.solve(verbose=True))

print(yi.value)

## cvxpy problem 

With variance

In [ ]:
def eval_fim(ya, yb, yc):
     
    fim = sum(yy1*FIM_set[idx] for idx, yy1 in enumerate(ya))+sum(yy2*FIM_set[idx+9]*0.25 for idx, yy2 in enumerate(yb)) + sum(yy3*FIM_set[idx+18]*0.25 for idx, yy3 in enumerate(yc))
                
    return fim 

def a_opt(yi):
    fim = eval_fim(yi)
    #return cp.matrix_frac(np.identity(fim.shape==[0]), fim)
    return cp.trace(fim)
    
def d_opt(ya, yb, yc):
    fim = eval_fim(ya, yb, yc)
    return cp.log_det(fim)

def e_opt(yi):
    fim = eval_fim(yi)
    return -cp.lambda_min(fim)

In [ ]:
no_res_unit = 9

ya = cp.Variable(no_res_unit, nonneg=True)
yb = cp.Variable(no_res_unit, nonneg=True)
yc = cp.Variable(no_res_unit, nonneg=True)

p_cons = [cp.sum(ya*10) + cp.sum(yb*6) + cp.sum(yc*6) <= 80]

for i in ya:
    p_cons += [i<=1]
for j in yb:
    p_cons += [j<=1]
for d in yc:
    p_cons += [d<=1]
    
obj = cp.Maximize(d_opt(ya, yb, yc))

problem = cp.Problem(obj, p_cons)



In [ ]:
print('First time: Optimal value:', problem.solve(verbose=True, max_iters=150000))

print('Second time: Optimal value:', problem.solve(verbose=True, warm_start = True))

In [ ]:
print(ya.value)
print(yb.value)
print(yc.value)

## CVXPY problem - with covariance 

### Continuous 

In [4]:
# variance 
variance = [1, # var_CA^2
            4, # var_CB^2
            8  # var_CC^2
           ]
covariance = [ 0.1, # cov_CA_CB | ti, 
               0.1, # cov_CA_CC | ti, 
               0.5 # cov_CB_CC | ti
]

# construct Sigma
sigma = np.zeros((27, 27))

for i in range(27):
    #for j in range(27):
        
        #if i==j:
    if i<9:
        sigma[i,i]=variance[0]
        sigma[i,i+9]=covariance[0]
        sigma[i,i+18]=covariance[1]
    elif 9<= i < 18:
        sigma[i,i]=variance[1]
        sigma[i,i-9]=covariance[0]
        sigma[i,i+9]=covariance[2]
    elif i>=18:
        sigma[i,i]=variance[2]
        sigma[i,i-18]=covariance[1]
        sigma[i,i-9]=covariance[2]
                
            
# check eigen values to make sure it's PSD 
print(np.linalg.eigvals(sigma))



[0.99566752 8.06331241 3.94102007 8.06331241 8.06331241 3.94102007
 3.94102007 0.99566752 0.99566752 0.99566752 0.99566752 3.94102007
 8.06331241 8.06331241 8.06331241 3.94102007 0.99566752 8.06331241
 3.94102007 0.99566752 0.99566752 8.06331241 0.99566752 8.06331241
 3.94102007 3.94102007 3.94102007]


In [ ]:
sigma_inv = np.linalg.inv(sigma)

In [ ]:
for i in range(9):
    for j in range(9):
        if sigma_inv[i,j] != sigma_inv[j,i]:
            print('wrong')
            
#print(sigma_inv)

print(np.linalg.eigvals(sigma_inv))
# verify: 

Q_array = np.asarray(Q_stack)

fim = Q_array.T@sigma_inv@Q_array

print(fim)
print('trace:', np.trace(fim))
print('det:', np.linalg.det(fim))
print('logdet:', np.log(np.linalg.det(fim)))
print('eig:', np.linalg.eigvals(fim))

In [ ]:
# construct FIMs. Expect 27*27 FIMs
# No 
print(np.shape(Q_stack))

Q_matrix = []
for i in range(27):
    Q_matrix.append(np.asarray(Q_stack[i]).reshape((4,1)))
# Q: [CA|t1, ..., CA|tN, CB|ti, ..., CC|ti, ...]

FIM_collection = []

for i in range(9):
    # yCA, CA
    mat = sigma_inv[i,i]*Q_matrix[i]@Q_matrix[i].T
    FIM_collection.append(mat)

for i in range(9):
    # yCA, CB
    FIM_collection.append(sigma_inv[i,i+9]*Q_matrix[i]@Q_matrix[i+9].T)

for i in range(9):
    # YCA, CC
    FIM_collection.append(sigma_inv[i,i+18]*Q_matrix[i]@Q_matrix[i+18].T)

for i in range(9,18): 
    FIM_collection.append(sigma_inv[i,i]*Q_matrix[i]@Q_matrix[i].T)

for i in range(9,18): 
    FIM_collection.append(sigma_inv[i,i-9]*Q_matrix[i]@Q_matrix[i-9].T)

for i in range(9,18): 
    FIM_collection.append(sigma_inv[i,i+9]*Q_matrix[i]@Q_matrix[i+9].T)

for i in range(18, 27): 
    FIM_collection.append(sigma_inv[i,i]*Q_matrix[i]@Q_matrix[i].T)

for i in range(18, 27): 
    FIM_collection.append(sigma_inv[i,i-18]*Q_matrix[i]@Q_matrix[i-18].T)

for i in range(18, 27): 
    FIM_collection.append(sigma_inv[i,i-9]*Q_matrix[i]@Q_matrix[i-9].T)
        

print(np.shape(FIM_collection))

In [ ]:
def compute():
    #ya = [0.5, 0.238, 0, 0.26, 0, 0, 0, 0, 0]
    yb = [0, 0.21, 0, 0, 0, 0, 0, 0, 0.29] 
    yc = [0, 0.36, 0, 0, 0, 0, 0, 0, 0.14]
    ya = [0.5]+[1]*8
    yb = [0.5]+[1]*8
    yc = [0.5]+[1]*8
    yac = [0.5]+[1]*6 + [0,0]
    yab = [0.5]+[1]*8
    ybc = [0.5]+[1]*8

    print(ya)

    #print(FIM_collection[33])

    fim = sum(ya1*FIM_collection[idx] for idx, ya1 in enumerate(ya)) \
        + sum(yb1*FIM_collection[idx+27] for idx, yb1 in enumerate(yb)) \
        + sum(yc1*FIM_collection[idx+54] for idx, yc1 in enumerate(yc)) \
        + sum(yab1*FIM_collection[idx+9] for idx, yab1 in enumerate(yab)) \
        + sum(yab1*FIM_collection[idx+36] for idx, yab1 in enumerate(yab)) \
        + sum(yac1*FIM_collection[idx+18] for idx, yac1 in enumerate(yac)) \
        + sum(yac1*FIM_collection[idx+63] for idx, yac1 in enumerate(yac)) \
        + sum(ybc1*FIM_collection[idx+45] for idx, ybc1 in enumerate(ybc)) \
        + sum(ybc1*FIM_collection[idx+72] for idx, ybc1 in enumerate(ybc))

    return fim

fim = compute()
    
print(fim)
print('trace:', np.trace(fim))
print('det:', np.linalg.det(fim))
print('logdet:', np.log(np.linalg.det(fim)))
print('eig:', np.linalg.eigvals(fim))

In [ ]:
def eval_fim(ya, yb, yc, yab, yac, ybc):
     
    fim = sum(ya1*FIM_collection[idx] for idx, ya1 in enumerate(ya)) \
    + sum(yb1*FIM_collection[idx+27] for idx, yb1 in enumerate(yb)) \
    + sum(yc1*FIM_collection[idx+54] for idx, yc1 in enumerate(yc)) \
    + sum(yab1*FIM_collection[idx+9] for idx, yab1 in enumerate(yab)) \
    + sum(yab1*FIM_collection[idx+36] for idx, yab1 in enumerate(yab)) \
    + sum(yac1*FIM_collection[idx+18] for idx, yac1 in enumerate(yac)) \
    + sum(yac1*FIM_collection[idx+63] for idx, yac1 in enumerate(yac)) \
    + sum(ybc1*FIM_collection[idx+45] for idx, ybc1 in enumerate(ybc)) \
    + sum(ybc1*FIM_collection[idx+72] for idx, ybc1 in enumerate(ybc))
    
    return fim 

def a_opt(ya, yb, yc, yab, yac, ybc):
    fim = eval_fim(ya, yb, yc, yab, yac, ybc)
    return cp.trace(fim)
    
def d_opt(ya, yb, yc, yab, yac, ybc):
    fim = eval_fim(ya, yb, yc, yab, yac, ybc)
    return cp.log_det(fim)

def e_opt(ya, yb, yc, yab, yac, ybc):
    fim = eval_fim(ya, yb, yc, yab, yac, ybc)
    return -cp.lambda_min(fim)

In [ ]:
no_res_unit = 9

ya = cp.Variable(no_res_unit, nonneg=True)
yb = cp.Variable(no_res_unit, nonneg=True)
yc = cp.Variable(no_res_unit, nonneg=True)
yab = cp.Variable(no_res_unit, nonneg=True)
yac = cp.Variable(no_res_unit, nonneg=True)
ybc = cp.Variable(no_res_unit, nonneg=True)

p_cons = [cp.sum(ya*10) + cp.sum(yb*6) + cp.sum(yc*6) <= 100]

#p_cons = []

for i in ya:
    p_cons += [i<=1]
for j in yb:
    p_cons += [j<=1]
for d in yc:
    p_cons += [d<=1]

#for a in yab:
#    p_cons += [a<=1]
#for b in yac:
#    p_cons += [b<=1]
#for c in ybc:
#    p_cons += [c<=1]


#for x in yab:
p_cons += [yab[i] <= ya[i] for i, idx in enumerate(ya)]
p_cons += [yab[i] <= yb[i] for i, idx in enumerate(yb)]
p_cons += [yac[i] <= ya[i] for i, idx in enumerate(ya)]
p_cons += [yac[i] <= yc[i] for i, idx in enumerate(yc)]
p_cons += [ybc[i] <= yb[i] for i, idx in enumerate(yb)]
p_cons += [ybc[i] <= yc[i] for i, idx in enumerate(yc)]

p_cons += [ya[i]+yb[i]-1<=yab[i] for i, idx in enumerate(ya)]
p_cons += [ya[i]+yc[i]-1<=yac[i] for i, idx in enumerate(ya)]
p_cons += [yb[i]+yc[i]-1<=ybc[i] for i, idx in enumerate(yb)]

#p_cons += [yab@np.ones(9) <= ya]
#p_cons += [yab@np.ones(9) <= yb]
#p_cons += [yac@np.ones(9) <= ya]
#p_cons += [yac@np.ones(9) <= yc]
#p_cons += [ybc@np.ones(9) <= yb]
#p_cons += [ybc@np.ones(9) <= yc]

#p_cons += [ya@np.ones(9) + yb@np.ones(9) - np.ones(9) <= yab]
#p_cons += [ya@np.ones(9) + yc@np.ones(9) - np.ones(9) <= yac]
#p_cons += [yb@np.ones(9) + yc@np.ones(9) - np.ones(9) <= ybc]
    
obj = cp.Maximize(d_opt(ya, yb, yc, yab, yac, ybc))

problem = cp.Problem(obj, p_cons)


In [ ]:
print('First time: Optimal value:', problem.solve(verbose=True))

In [ ]:
print(ya.value)
print(yb.value)
print(yc.value)
print(yab.value)
print(yac.value)
print(ybc.value)

### tests

In [ ]:

test = [[1,0,0,5,0,0,9,0,0],
        [0,1,0,0,5,0,0,9,0],
        [0,0,1,0,0,5,0,0,9],
        [5,0,0,4,0,0,6,0,0],
        [0,5,0,0,4,0,0,6,0],
        [0,0,5,0,0,4,0,0,6],
        [9,0,0,6,0,0,4,0,0],
        [0,9,0,0,6,0,0,4,0],
        [0,0,9,0,0,6,0,0,4]]

test1 = np.asarray(test)

In [ ]:
print(np.linalg.inv(test1))